In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
name_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_list_with_name.db')
annual_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_eps_annual.db')
quarter_engine = create_engine('sqlite:///../../../dataset/us/us_ticker_eps_quarter.db')

In [28]:
df[df.eps_g1 < 1]

,ticker,eps_g1,eps_g2,eps_g3
42,ANET,0.516129,0.491228,0.275862
48,APH,0.288136,0.283019,0.275862
61,ARW,0.833688,0.831585,0.794449
109,BLD,0.774551,0.522574,0.241150
141,CASS,0.258078,0.237245,0.225000
161,CHRW,0.854167,0.601562,0.596330
162,CHTR,0.671174,0.552394,0.546312
175,CNXN,0.791375,0.730769,0.260417
211,DLNG,0.655303,0.438636,0.425000
214,DMTK,0.757165,0.742256,0.517952


In [6]:
print(df)

    ticker     eps_g1     eps_g2     eps_g3
0       AA   1.243618  51.838710  47.115385
1      AAL  21.666667   0.279188  -0.607843
2      AAT   4.100000   4.800000   1.362500
3      ACB   3.388235  -0.776119  -0.927536
4     ACCD   4.857143  -1.462366  -1.968750
..     ...        ...        ...        ...
953    YOU   2.000000       -inf       -inf
954    YRD   3.461538  -2.057143  10.000000
955    YVR   5.378571   5.303448   0.603448
956   ZEST   3.555556  -1.395349 -13.000000
957    ZIM   1.133041        inf        inf

[958 rows x 4 columns]


In [7]:
name_df = pd.read_sql('TOTAL', con=name_engine)
name_list = name_df.Symbol.to_list()

# Code

In [ ]:
# when target date equals to 0, means getting the lastest 
# when target date equals to 1, means getting the last quarter
def select_by_eps(targetdate = 0):
    pd.set_option('mode.chained_assignment', None)
    selected_df = pd.DataFrame(columns=['ticker','eps_g1','eps_g2','eps_g3'])
    for ticker in name_list:
        try:
            quarter_eps = pd.read_sql('{}'.format(ticker),con=quarter_engine)
            if len(quarter_eps) < targetdate + 7:
                print('{} data not enough'.format(ticker))
                continue
            targettime = quarter_eps.iloc[targetdate].datetime
            quarter_eps['yoy_growth'] = 0
            for i in range(len(quarter_eps)-4):
                quarter_eps.yoy_growth.iloc[i] = (quarter_eps.eps.iloc[i] - quarter_eps.eps.iloc[i+4]) / quarter_eps.eps.iloc[i+4]
            # fix the growth rate
            quarter_eps['fixed_yoy_growth'] = 0
            for i in range(len(quarter_eps)-1):
                quarter_eps.fixed_yoy_growth.iloc[i] = (quarter_eps.yoy_growth.iloc[i] + quarter_eps.yoy_growth.iloc[i+1]) / 2 
            # pick the ticker
            if quarter_eps.iloc[targetdate].yoy_growth > quarter_eps.iloc[targetdate+1].yoy_growth > quarter_eps.iloc[targetdate+2].yoy_growth > 0.2 or quarter_eps.iloc[targetdate].yoy_growth > 1:
                eps_g1 = quarter_eps.iloc[targetdate].yoy_growth
                eps_g2 = quarter_eps.iloc[targetdate+1].yoy_growth
                eps_g3 = quarter_eps.iloc[targetdate+2].yoy_growth
                temp = pd.DataFrame([[ticker,eps_g1,eps_g2,eps_g3]],columns=['ticker','eps_g1','eps_g2','eps_g3'])
                selected_df = pd.concat([selected_df,temp])
                print('{} selected'.format(ticker))
            elif quarter_eps.iloc[targetdate].fixed_yoy_growth > quarter_eps.iloc[targetdate+1].fixed_yoy_growth > quarter_eps.iloc[targetdate+2].fixed_yoy_growth > 0.2 or quarter_eps.iloc[targetdate].fixed_yoy_growth > 1:
                eps_g1 = quarter_eps.iloc[targetdate].fixed_yoy_growth
                eps_g2 = quarter_eps.iloc[targetdate+1].fixed_yoy_growth
                eps_g3 = quarter_eps.iloc[targetdate+2].fixed_yoy_growth
                temp = pd.DataFrame([[ticker,eps_g1,eps_g2,eps_g3]],columns=['ticker','eps_g1','eps_g2','eps_g3'])
                selected_df = pd.concat([selected_df,temp])
                print('{} selected'.format(ticker))
            else:
                print('{} not selected'.format(ticker))
        except:
            print('{} not exsit'.format(ticker))
    # 删除inf
    selected_df.replace([np.inf, -np.inf], np.nan,inplace=True)
    selected_df.dropna(axis=0, inplace=True)
    selected_df.index = range(len(selected_df))
    selected_df.to_csv('./results/selected_ticker_by_eps_{}.csv'.format(targettime),index=None)
    # selected_df.to_csv('selected_ticker_by_eps.csv',index=None)
select_by_eps()

In [23]:
sl = [1,2,3,4]
df = pd.DataFrame(sl,columns=['symbol'])
df
selected_df = pd.DataFrame([['a',1,2,3]],columns=['ticker','eps1','eps2','eps3'])
selected_df

,ticker,eps1,eps2,eps3
0,a,1,2,3


# sample

In [2]:
pd.set_option('mode.chained_assignment', None)
 # quarter
quarter_eps = pd.read_sql('{}'.format('AAPL'),con=quarter_engine)
# quarter_eps = quarter_eps.head(8)
# if len(quarter_eps) == 8:
quarter_eps['yoy_growth'] = 0
for i in range(len(quarter_eps)-4):
    quarter_eps.yoy_growth.iloc[i] = (quarter_eps.eps.iloc[i] - quarter_eps.eps.iloc[i+4]) / quarter_eps.eps.iloc[i+4]
# store the data
quarter_eps
# fix the growth rate
quarter_eps['fixed_yoy_growth'] = 0
for i in range(len(quarter_eps)-1):
    quarter_eps.fixed_yoy_growth.iloc[i] = (quarter_eps.yoy_growth.iloc[i] + quarter_eps.yoy_growth.iloc[i+1]) / 2 
quarter_eps
# pick the ticker
if quarter_eps.iloc[0].yoy_growth > quarter_eps.iloc[1].yoy_growth > quarter_eps.iloc[2].yoy_growth > 0.2 or quarter_eps.iloc[0].yoy_growth > 1:
    print('selected')
elif quarter_eps.iloc[0].yoy_growth > quarter_eps.iloc[1].fixed_yoy_growth > quarter_eps.iloc[2].fixed_yoy_growth > 0.2 or quarter_eps.iloc[0].fixed_yoy_growth > 1:
    print('selected')
else:
    print('not selected')

# if str(quarter_eps.iloc[0].datetime)[:4] != '2022':
#     continue
# elif quarter_eps.iloc[:3].eps.min() < 0:
#     continue
# else:
#     updatetime = quarter_eps.iloc[0].datetime
#     length = len(quarter_eps)
#     quarter_eps['q_chg_rate'] = 0.0
#     for i in range(length-1):
#         quarter_eps['q_chg_rate'].iloc[i]  = round(((quarter_eps.iloc[i].eps - quarter_eps.iloc[i+1].eps) / quarter_eps.iloc[i+1].eps),3).copy()
#     quarter_chg_rate = round(quarter_eps.iloc[:3].q_chg_rate.mean(),4)
#     # print('{} eps: {}'.format(ticker, quarter_chg_rate))

not selected


In [4]:
quarter_eps.iloc[1].datetime

'2022-03-31'